In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from pandas.plotting import scatter_matrix

import plotly
import plotly.offline as py
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
from mestimator import MEstimator

In [3]:
df_train = pd.read_csv('data_train.csv')
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1.618982,1.541605,-0.251879,-0.842436,0.184519,0.937082,0.731000,1.361556,-0.326238,0.055676,...,-1.443217,-0.756352,0.816454,0.750445,-0.455947,1.189622,-1.690617,-1.356399,-1.232435,1.0
1,-0.544439,-0.668172,0.007315,-0.612939,1.299748,-1.733096,-0.983310,0.357508,-1.613579,1.470714,...,-0.549746,-0.940046,-0.827932,0.108863,0.507810,-0.862227,1.249470,-0.079611,-0.889731,1.0
2,-0.881798,0.018639,0.237845,0.013549,-1.635529,-1.044210,0.613039,0.736205,1.026921,-1.432191,...,0.366093,-0.331777,-0.689218,2.034608,-0.550714,0.750453,-1.306992,0.580573,-1.104523,1.0
3,0.690121,0.686890,-1.566688,0.904974,0.778822,0.428233,0.108872,0.028284,-0.578826,-1.199451,...,0.369164,1.876573,-0.376903,1.831936,0.003017,-0.076023,0.003958,-0.185014,-2.487152,0.0
4,-1.704651,-1.136261,-2.973315,0.033317,-0.248889,-0.450176,0.132428,0.022214,0.317368,-0.752414,...,0.095139,-0.423715,-1.185984,-0.365462,-1.271023,1.586171,0.693391,-1.958081,-0.134801,1.0


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 21 columns):
0     1100 non-null float64
1     1100 non-null float64
2     1100 non-null float64
3     1100 non-null float64
4     1100 non-null float64
5     1100 non-null float64
6     1100 non-null float64
7     1100 non-null float64
8     1100 non-null float64
9     1100 non-null float64
10    1100 non-null float64
11    1100 non-null float64
12    1100 non-null float64
13    1100 non-null float64
14    1100 non-null float64
15    1100 non-null float64
16    1100 non-null float64
17    1100 non-null float64
18    1100 non-null float64
19    1100 non-null float64
20    1100 non-null float64
dtypes: float64(21)
memory usage: 180.5 KB


In [5]:
df_train.describe()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
count,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,...,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000
mean,-0.044329,-0.005963,-0.064988,-0.004037,0.042393,-0.058549,0.074697,0.154462,-0.019384,0.048542,...,-0.115676,0.153875,-0.001760,0.105399,-0.100943,0.032874,0.043116,-0.050098,0.002567,0.487273
std,2.049145,1.862960,1.998252,2.047336,1.904181,1.991460,1.947067,1.961074,2.043635,1.945284,...,2.074375,1.933526,1.936262,1.969506,1.985764,1.877694,1.981002,2.025241,2.034693,0.500065
min,-9.826618,-9.780256,-9.946673,-9.873077,-9.699020,-9.939023,-9.632021,-9.737222,-9.987724,-9.888273,...,-9.870625,-9.029958,-9.918501,-9.544553,-9.783310,-9.711194,-9.887228,-9.993739,-9.755028,0.000000
25%,-0.811175,-0.766298,-0.732832,-0.787628,-0.706137,-0.747007,-0.745667,-0.654544,-0.781310,-0.727804,...,-0.792120,-0.645971,-0.710151,-0.681649,-0.793342,-0.692138,-0.689272,-0.787835,-0.790919,0.000000
50%,-0.090252,-0.019721,0.019931,-0.024077,0.023195,-0.021290,0.071588,0.093202,-0.004351,-0.010843,...,-0.071567,0.035345,0.017473,0.034528,-0.077669,0.050182,0.006734,-0.027256,-0.049497,0.000000
75%,0.683786,0.751208,0.700758,0.721515,0.813496,0.652242,0.808881,0.844679,0.735884,0.721748,...,0.693668,0.810641,0.766504,0.858886,0.705434,0.755980,0.762986,0.734604,0.739056,1.000000
max,9.918598,9.258123,9.997424,9.871701,9.521661,9.721049,9.780127,9.756564,9.853715,9.554631,...,9.741268,9.983469,9.859751,9.656089,9.819445,9.958788,9.770962,9.813015,9.971586,1.000000


In [6]:
df_train['20'] = df_train['20'].astype(int)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1.618982,1.541605,-0.251879,-0.842436,0.184519,0.937082,0.731000,1.361556,-0.326238,0.055676,...,-1.443217,-0.756352,0.816454,0.750445,-0.455947,1.189622,-1.690617,-1.356399,-1.232435,1
1,-0.544439,-0.668172,0.007315,-0.612939,1.299748,-1.733096,-0.983310,0.357508,-1.613579,1.470714,...,-0.549746,-0.940046,-0.827932,0.108863,0.507810,-0.862227,1.249470,-0.079611,-0.889731,1
2,-0.881798,0.018639,0.237845,0.013549,-1.635529,-1.044210,0.613039,0.736205,1.026921,-1.432191,...,0.366093,-0.331777,-0.689218,2.034608,-0.550714,0.750453,-1.306992,0.580573,-1.104523,1
3,0.690121,0.686890,-1.566688,0.904974,0.778822,0.428233,0.108872,0.028284,-0.578826,-1.199451,...,0.369164,1.876573,-0.376903,1.831936,0.003017,-0.076023,0.003958,-0.185014,-2.487152,0
4,-1.704651,-1.136261,-2.973315,0.033317,-0.248889,-0.450176,0.132428,0.022214,0.317368,-0.752414,...,0.095139,-0.423715,-1.185984,-0.365462,-1.271023,1.586171,0.693391,-1.958081,-0.134801,1


## Detect and delete outliers using m-estimator

In [7]:
# get the hyperparameters: HERE THE N IS ALREADY KNOWN
n = 10000
p = 20

In [9]:
# store the numpy array representation of the dataframe
X = df_train.drop(['20'], axis=1).values

me = MEstimator()
# estimated location of inliers
loc_estimated = me.loc_estimator(X)
scale_estimated = me.scale_estimator(X)
X = X - loc_estimated
X_norm = np.linalg.norm(X, axis=1)

# use M-estimator of scale to compute the criteria of selecting inliers
T = 4 * np.sqrt((np.log(p) + np.log(n))) * np.max(scale_estimated)
inliner_marker = np.where(X_norm < T)[0]

# select inliers
df_train_in = df_train.iloc[inliner_marker]

/Users/yaweili/PycharmProjects/RBLR/mestimator.py:68: RuntimeWarning:

divide by zero encountered in true_divide

/Users/yaweili/PycharmProjects/RBLR/mestimator.py:68: RuntimeWarning:

invalid value encountered in true_divide

/Users/yaweili/PycharmProjects/RBLR/mestimator.py:37: RuntimeWarning:

divide by zero encountered in true_divide

/Users/yaweili/PycharmProjects/RBLR/mestimator.py:37: RuntimeWarning:

invalid value encountered in true_divide

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning:

invalid value encountered in reduce



In [10]:
# set X,y training data for LR regressor
clf = LogisticRegression(solver='liblinear')
X_train = df_train_in.drop(['20'], axis=1)
y_train = df_train_in['20']

# train the regressor
clf.fit(X_train, y_train)

# load and prepare the test data
df_test = pd.read_csv('data_test.csv')
X_test = df_test.drop(['20'], axis=1)
y_test = df_test['20'].astype(int)

score = clf.score(X_test, y_test)
print('score: %f' % score)

y_predict = clf.predict(X_test)
print(classification_report(y_test, y_predict,))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

invalid value encountered in less

